In [5]:
!pip install requests

You are using pip version 18.0, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


##Data vizualization

In [6]:
!pip install azure-cognitiveservices-vision-customvision

You are using pip version 18.0, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [15]:
from azure.cognitiveservices.vision.customvision.training import training_api
from azure.cognitiveservices.vision.customvision.training.models import ImageUrlCreateEntry

# Replace with a valid key
training_key = "ad4f1a6056754218be454e19740fc7d6"
prediction_key = "67576df71b704a278685868494d0229c"

trainer = training_api.TrainingApi(training_key)

# Create a new project
print ("Creating project...")
project = trainer.create_project("Challenge-1")

Creating project...


In [16]:
# Make two tags in the new project
hemlock_tag = trainer.create_tag(project.id, "hardshell-jacket")
cherry_tag = trainer.create_tag(project.id, "insulated-jacket")

In [17]:
import os
hemlock_dir = "data/gear_images/hardshell_jackets"
for image in os.listdir(os.fsencode("data/gear_images/hardshell_jackets")):
    with open(hemlock_dir + "/" + os.fsdecode(image), mode="rb") as img_data: 
        trainer.create_images_from_data(project.id, img_data, [ hemlock_tag.id ])

cherry_dir = "data/gear_images/insulated_jackets"
for image in os.listdir(os.fsencode("data/gear_images/insulated_jackets")):
    with open(cherry_dir + "/" + os.fsdecode(image), mode="rb") as img_data: 
        trainer.create_images_from_data(project.id, img_data, [ cherry_tag.id ])

In [18]:
import time

print ("Training...")
iteration = trainer.train_project(project.id)
while (iteration.status != "Completed"):
    iteration = trainer.get_iteration(project.id, iteration.id)
    print ("Training status: " + iteration.status)
    time.sleep(1)

# The iteration is now trained. Make it the default project endpoint
trainer.update_iteration(project.id, iteration.id, is_default=True)
print ("Done!")

Training...
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Completed
Done!


In [31]:
from azure.cognitiveservices.vision.customvision.prediction import prediction_endpoint
from azure.cognitiveservices.vision.customvision.prediction.prediction_endpoint import models

# Now there is a trained endpoint that can be used to make a prediction

predictor = prediction_endpoint.PredictionEndpoint(prediction_key)

test_img_url = "https://cdn.shopify.com/s/files/1/0108/7802/products/manning-9-1205413.jpg?v=1506516432"
results = predictor.predict_image_url(project.id, iteration.id, url=test_img_url)


# Display the results.
for prediction in results.predictions:
    print ("\t" + prediction.tag_name + ": {0:.2f}%".format(prediction.probability * 100))
    print('Solution: insulated')


	insulated-jacket: 99.97%
Solution: insulated
	hardshell-jacket: 0.01%
Solution: insulated


In [23]:
project.id

'3bd80196-bf67-42e6-8ea0-1512eb7ebd78'